# Python Package Requirements

In [1]:
# Requirements:
## - Install the following python packages:
###  - `pip3 install cubes sqlalchemy flask`

# Create Schema and Load Data

In [2]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
engine = create_engine("sqlite:///data.sqlite")
create_table_from_csv(engine,
                     "../data/Table_E.csv",
                     table_name="nyc_restaurants",
                     fields=[
                        ("name", "string"),
                        ("address", "string"),
                        ("zipcode", "string"),
                        ("cuisine", "string"),
                        ("price", "string"),
                        ("rating", "string"),
                        ("review_count", "integer"),
                        ("violation_code", "string"),
                        ("critical_flag", "string"),
                        ("grade", "string"),
                        ("median_household_income", "integer"),
                        ("median_real_estate_value", "integer"),
                        ("population_density", "integer"),
                        ("cost_of_living", "integer"),
                        ("population", "integer"),
                        ("neighborhood", "string"),
                        ("borough", "string")
                     ],
                     create_id=True)
print('Created schema and loaded data.')

Created schema and loaded data.


# Create OLAP Cube

In [3]:
from cubes import Workspace
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("olap_model.json")
browser = workspace.browser("nyc_restaurants")
print('Created OLAP cube.')

Created OLAP cube.


# Aggregation by borough

In [4]:
result = browser.aggregate(drilldown=["area:borough"])
print('Aggregation results:')
print("Borough, Number of Restaurants")
for record in result:
    print(record['area.borough'], ",",record['num_restaurants'])

Aggregation results:
Borough, Number of Restaurants
Bronx , 519
Brooklyn , 1846
Manhattan , 1821
Queens , 1078
Staten Island , 296


# Aggregation by neighborhood

In [5]:
result = browser.aggregate(drilldown=["area:neighborhood"])
print('Aggregation results:') 
print("Neighborhood, Number of Restaurants")
for record in result:
    print(record['area.neighborhood'], ",",record['num_restaurants'])

Aggregation results:
Neighborhood, Number of Restaurants
Bronx Park and Fordham , 103
Central Bronx , 23
High Bridge and Morrisania , 36
Hunts Point and Mott Haven , 38
Kingsbridge and Riverdale , 62
Northeast Bronx , 77
Southeast Bronx , 180
Borough Park , 136
Bushwick and Williamsburg , 172
Canarsie and Flatlands , 82
Central Brooklyn , 239
East New York and New Lots , 54
Flatbush , 105
Greenpoint , 207
Northwest Brooklyn , 442
Southern Brooklyn , 191
Southwest Brooklyn , 133
Sunset Park , 85
Battery Park , 3
Central Harlem , 88
Chelsea and Clinton , 260
East Harlem , 83
Gramercy Park and Murray Hill , 181
Greenwich Village and Soho , 273
Inwood and Washington Heights , 201
Lower East Side , 212
Lower Manhattan , 206
Upper East Side , 120
Upper West Side , 194
Central Queens , 31
Jamaica , 105
North Queens , 185
Northeast Queens , 67
Northwest Queens , 172
Southeast Queens , 71
Southwest Queens , 97
West Central Queens , 129
West Queens , 221
Mid-Island , 55
Port Richmond , 57
South 

# Aggregation by restaurant price type

In [6]:
result = browser.aggregate(drilldown=["price"])
print('Aggregation results:')
print("Price Type, Number of Restaurants")
for record in result:
    print(record['price'], ",", record['num_restaurants'])

Aggregation results:
Price Type, Number of Restaurants
$ , 2280
$$ , 2930
$$$ , 300
$$$$ , 50


# Aggregation by restaurant rating 

In [7]:
result = browser.aggregate(drilldown=["rating"])
print('Aggregation results:')
print("Rating, Number of Restaurants")
for record in result:
    print(record['rating'], ",", record['num_restaurants'])

Aggregation results:
Rating, Number of Restaurants
1.0 , 43
1.5 , 85
2.0 , 168
2.5 , 379
3.0 , 780
3.5 , 1424
4.0 , 1825
4.5 , 726
5.0 , 130


# Aggregation by critical health violation

In [8]:
result = browser.aggregate(drilldown=["critical_flag"])
print('Aggregation results:')
print("Health Violation, Number of Restaurants")
for record in result:
    print(record['critical_flag'], ",", record['num_restaurants'])

Aggregation results:
Health Violation, Number of Restaurants
Critical , 2991
Not Critical , 2569


# Aggregation by health violation grade

In [10]:
result = browser.aggregate(drilldown=["grade"])
print('Aggregation results:')
print("Health Violation Grade, Number of Restaurants")
for record in result:
    print(record['grade'], ",", record['num_restaurants'])

Aggregation results:
Health Violation Grade, Number of Restaurants
A , 4881
B , 422
C , 46
P , 10
Z , 201
